<a href="https://colab.research.google.com/github/Firojpaudel/GenAI-Chronicles/blob/main/BERTs/BERT_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Trying to Implement BERT**: _Using Huggingface_ 🤗

---

I'm learning from [The Official Hugging Face Transformer Docs](https://huggingface.co/docs/transformers/index). And I'll be using PyTorch the entire time.

In [ ]:
##@ First lets install the huggingface trannsformers, datasets, evaluate and accelerate
! pip install transformers datasets evaluate accelerate

### Getting Started:

#### **A. Pipeline**

**Important Catalogue before starting:**

---


| **Task**                     | **Description**                                                                                              | **Modality**    | **Pipeline identifier**                       |
|------------------------------|--------------------------------------------------------------------------------------------------------------|-----------------|-----------------------------------------------|
| Text classification          | assign a label to a given sequence of text                                                                   | NLP             | pipeline(task=“sentiment-analysis”)           |
| Text generation              | generate text given a prompt                                                                                 | NLP             | pipeline(task=“text-generation”)              |
| Summarization                | generate a summary of a sequence of text or document                                                         | NLP             | pipeline(task=“summarization”)                |
| Image classification         | assign a label to an image                                                                                   | Computer vision | pipeline(task=“image-classification”)         |
| Image segmentation           | assign a label to each individual pixel of an image (supports semantic, panoptic, and instance segmentation) | Computer vision | pipeline(task=“image-segmentation”)           |
| Object detection             | predict the bounding boxes and classes of objects in an image                                                | Computer vision | pipeline(task=“object-detection”)             |
| Audio classification         | assign a label to some audio data                                                                            | Audio           | pipeline(task=“audio-classification”)         |
| Automatic speech recognition | transcribe speech into text                                                                                  | Audio           | pipeline(task=“automatic-speech-recognition”) |
| Visual question answering    | answer a question about the image, given an image and a question                                             | Multimodal      | pipeline(task=“vqa”)                          |
| Document question answering  | answer a question about a document, given an image and a question                                            | Multimodal      | pipeline(task="document-question-answering")  |
| Image captioning             | generate a caption for a given image                                                                         | Multimodal      | pipeline(task="image-to-text")                |


In [1]:
from transformers import pipeline

classifier= pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [2]:
classifier("Hey! so this way we can pass the value to classifier and its super easy. I'm liking this!")

[{'label': 'POSITIVE', 'score': 0.9996222257614136}]

In [3]:
#@ Testing for negative sentiment
classifier("You dummy!")

[{'label': 'NEGATIVE', 'score': 0.9874186515808105}]

Likewise, if we have more than one inputs, we can pass inputs as lists to the `pileline()` and that will return the list of dictionaries.


In [4]:
results = classifier(["You look beautiful", "You ugly hag!"])
for result in results:
  print(f"label: {result['label']}, score: {result['score']}")

label: POSITIVE, score: 0.9998769760131836
label: NEGATIVE, score: 0.9993403553962708


Also the `pipeline()` can iterate over the entire dataset.

In [5]:
import torch

speech_recognizer = pipeline("automatic-speech-recognition", model= "facebook/wav2vec2-base-960h")

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

Device set to use cpu


In [8]:
from datasets import load_dataset, Audio

dataset= load_dataset("PolyAI/minds14", name="en-US", split="train")

The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


MInDS-14.zip:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

What we are dong in above code snippet is that: we are loading the specific "PolyAIs MINDS-14" dataset from the Huggingface hub.

Likewise `en-us` as name specifies the particular subset or config of the dataset. In this case, it loads the English(US) subset of the dataset.


Then there comes `split="train"` which specifies the split of the dataset to load.

In [9]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))

More explanations:

1. The `cast_column` Method:
Its similar to type_casting that we used to do in basics of python. Here, what it does is, it modifies the "audio" column to use the `Audio` type with a specified sampling rate.

_Why cast_column?_

- So that all the data are standardized  to the desired format.

In [12]:
result = speech_recognizer(dataset[:2]["audio"])
print([d["text"] for d in result])

['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE"]
